In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [33]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()
    
# List all unique characters that occurs in the input text
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)

In [35]:
# mapping from characters to integers for encoding
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # Take a string and output a list of integers
decode = lambda i : ''.join([itos[c] for c in i]) # Take a list of integers and output a list of string

In [36]:
# encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(shakespeare), dtype=torch.long)

In [37]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [38]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will be process in parallel?
block_size = 8 # What is the maximum context length for predictions?

In [39]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device=DEVICE), y.to(device=DEVICE)

In [40]:
class Head(nn.Module):
    """ One head of self-attention """
    
    def __init__(self, n_embd, block_size, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril((torch.ones(block_size, block_size))))
    
    def forward(self, x):
        B, T, C = x.shape
        
        # Compute the attention score using k and q 
        k = self.key(x)
        q = self.query(x)
        w = q @ k.transpose(-2, -1) * C ** -0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        
        # weight aggregation of the values using the attention score
        v = self.value(x)
        out = w @ v
        
        return out

In [41]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, n_embd, block_size, head_size, n_head):
        super().__init__()
        self.head = nn.ModuleList([Head(n_embd, block_size, head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        # Use multiple heads of self-attention 
        # Concatanate each output alone the embedding dimension (B, T ,C), that is merged alone C
        out = torch.cat([h(x) for h in self.head], dim=-1)
        out = self.proj(out)
        return out

In [42]:
class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embd) -> None:
        super().__init__()
        self.net = nn.Sequential(
                    nn.Linear(n_embd, 4 * n_embd),
                    nn.ReLU(),
                    nn.Linear(4 * n_embd, n_embd),
        )
        
    def forward(self, x):
        return self.net(x)

In [43]:
class AttnBlock(nn.Module):
    def __init__(self, n_embd, block_size, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head
        self.attn = MultiHeadAttention(n_embd, block_size, head_size, n_head)
        self.ffn = FeedForward(n_embd)
        
        # ------------------modified--------------------------------------
        self.ln = nn.LayerNorm(n_embd)
        # ------------------modified--------------------------------------
    
    def forward(self, x):
        # ------------------modified--------------------------------------
        # Use the residual connection with layernorm
        x = x + self.attn(self.ln(x))
        x = x + self.ffn(self.ln(x))
        # ------------------modified--------------------------------------

        return x

In [44]:
# A modified neural network for bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, block_size, n_head) -> None:
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # Use attention block consisting of the multi-head attention and feedforward layer
        self.attn_blocks = nn.Sequential(
                AttnBlock(n_embd, block_size, n_head),
                AttnBlock(n_embd, block_size, n_head),
                AttnBlock(n_embd, block_size, n_head),
                nn.LayerNorm(n_embd),
        )

        self.ln_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE))
        x = tok_emb + pos_emb

        # ------------------modified--------------------------------------
        # use the attention blocks
        x = self.attn_blocks(x)
        # ------------------modified--------------------------------------

        logits = self.ln_head(x)            
        
        if targets is None: # for generation without providing target
            loss = None
        else:
            # Reshape the logits tensor to meet definition of the cross_entropy function in Pytorch
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Concatenate B, T
            targets = targets.view(B*T)    # Same reshaping to the target tensor
            loss = F.cross_entropy(logits, targets) # calculate the loss
        
        return logits, loss
    
    def generation(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens to meet position embedding range
            idx_crop = idx[:, -self.block_size:]
            
            # get the predictions
            logits, _ = self(idx_crop)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx

In [45]:
model = BigramLanguageModel(vocab_size, n_embd=32, block_size=block_size, n_head=4)
model.to(device=DEVICE)

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (position_embedding_table): Embedding(8, 32)
  (attn_blocks): Sequential(
    (0): AttnBlock(
      (attn): MultiHeadAttention(
        (head): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=32, out_features=8, bias=False)
            (query): Linear(in_features=32, out_features=8, bias=False)
            (value): Linear(in_features=32, out_features=8, bias=False)
          )
        )
        (proj): Linear(in_features=32, out_features=32, bias=True)
      )
      (ffn): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=32, out_features=128, bias=True)
          (1): ReLU()
          (2): Linear(in_features=128, out_features=32, bias=True)
        )
      )
      (ln): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (1): AttnBlock(
      (attn): MultiHeadAttention(
        (head): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_fe

In [46]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [47]:
from tqdm import tqdm

batch_size = 32
epochs = 10000
losses = []
for e in tqdm(range(epochs)):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

100%|██████████| 10000/10000 [01:54<00:00, 87.19it/s]


In [48]:
print(loss.item())

1.991146445274353


In [50]:
print(decode(model.generation(idx=torch.zeros((1, 1), dtype=torch.long, device=DEVICE), max_new_tokens=500)[0].tolist()))


Men panss and grition I what you so a cemost.

DUKE VINCENTIO:
Was mayst abjess fincle to one.

Thuch fripy:
Which might. 
CEOMPARK:
Make therseled and he moveto imdians
Less's colmeed; bose so not his re'er lates:
Whit Clonvers! I know, look tone.

xo leard: he and that hichbur's am know it bump nemided so lacknow a speak their subs?
Their hath bronain, dree?
Fyraiter,
How time done no stee summondip you ou what thang the silst, Bloost mans as hear the biddraw thine.

EWIZABETER:
S;
To know our
